## Load data

In [ ]:
load('MEGdata.mat')  

## Split data into training/testing

In [ ]:
N = size(VideoClass,1);     % Number of samples
Ntrain = 400;

idx = randperm(N);      
trndata = double( MEGdata(idx(1:Ntrain),:,:));
trnclass = double ( VideoClass(idx(1:Ntrain)));

NumOfClasses = length(unique(trnclass));

testdata = double( MEGdata(idx(Ntrain+1:end),:,:) );
testclass = double( VideoClass(idx(Ntrain+1:end)));

## Feature extraction
- Mean and variance from 204 channels

In [ ]:
trn_feature = [mean(trndata,3) var(trndata,0,3)];
test_feature = [mean(testdata,3) var(testdata,0,3)];

## Logistic Regression
- Matlab doesn't seem work
 * B = mnrfit(trn_feature, trnclass);
 * pihat_LR = mnrval(B,test_feature);
- Thus, we use glmnet toolbox

In [ ]:
fitglmnet = cvglmnet(trn_feature,trnclass,'multinomial');
pihat_LR = glmnetPredict(fitglmnet.glmnet_fit,test_feature,fitglmnet.lambda_min,'response');
[~,LR_predicted_class] = max(pihat_LR,[],2);
LR_pred_rate = sum(testclass == LR_predicted_class)./length(testclass)

## Neural Network

In [ ]:
% One-hot encoder
trnclasshat = zeros(Ntrain,NumOfClasses);
for i=1:Ntrain
    trnclasshat(i,trnclass(i)) = 1;
end
net = patternnet(10);
net = train(net,trn_feature',trnclasshat');
%view(net)
pihat_NN = net(test_feature');
pihat_NN = pihat_NN';
[~,NN_predicted_class] = max(pihat_NN,[],2);
NN_pred_rate = sum(testclass == NN_predicted_class)./length(testclass)

## Naive-bayesian (NB) classifier with Gaussian data distribution

In [ ]:
nb = NaiveBayes.fit(trn_feature, trnclass);
pihat_NB = posterior(nb,test_feature);
NB_predicted_class = predict(nb, test_feature);
NB_pred_rate = sum(testclass == NB_predicted_class)./length(testclass)

## Combining classifiers by averaging

In [ ]:
% sum
pihat_sum = pihat_NN + pihat_NB+ pihat_LR;
pihat_sum = bsxfun(@rdivide, pihat_sum, sum(pihat_sum,2));
[~,predicted_class_sum] = max(pihat_sum,[],2);
% pred_rate_sum = sum(testclass == predicted_class_sum)./length(testclass)

In [ ]:
% product
pihat_product = pihat_NN .* pihat_NB .*pihat_LR;
pihat_product = bsxfun(@rdivide, pihat_product, sum(pihat_product,2));
[~,predicted_class_product] = max(pihat_product,[],2);
% pred_rate_product = sum(testclass == predicted_class_product)./length(testclass)

In [ ]:
% max
pihat_max = max(cat(3,pihat_NN,pihat_NB,pihat_LR),[],3);
pihat_max = bsxfun(@rdivide, pihat_max, sum(pihat_max,2));
[~,predicted_class_max] = max(pihat_max,[],2);
% pred_rate_max = sum(testclass == predicted_class_max)./length(testclass)

In [ ]:
SumAccuracy = mean(predicted_class_sum==testclass)
ProductAccuracy = mean(predicted_class_product==testclass)
MaxAccuracy = mean(predicted_class_max==testclass)